# Data Cleaning

## Read File

In [1]:
# Imports
import pandas as pd
telemetry = pd.read_csv('./telemetry.etr', delim_whitespace=True)

## Normalise Data
In order to anonymise the data, we must normalise the Easting and Northing values. The following formula is used:
$$
z_i = \frac{x_i - \text{min}(x)}{\text{max}(x)-\text{min}(x)}
$$

In [3]:
# Normalisation function
def normalise(column: str) -> float:
	min = telemetry[column].min()
	max = telemetry[column].max()
	x = telemetry[column]
	return (x - min) / (max - min)

telemetry['Easting'] = normalise('Easting')
telemetry['Northing'] = normalise('Northing')
telemetry['WaterDepth'] = normalise('WaterDepth')

In [4]:
telemetry.head(5)

,Date,Time,Easting,Northing,WaterDepth,Roll,Pitch,Heading,Tide
0,20-02-27,20:50:47.502,0.158187,0.410159,0.619607,-6.3,2.0,19.9,0.0
1,20-02-27,20:50:47.502,0.158187,0.410159,0.038946,-6.3,2.0,19.9,0.0
2,20-02-27,20:50:49.416,0.534407,0.535898,0.038946,-5.0,3.1,22.8,0.0
3,20-02-27,20:50:49.416,0.534311,0.535924,0.042081,-5.0,3.1,22.8,0.0
4,20-02-27,20:50:49.416,0.533734,0.536254,0.046682,-5.0,3.1,22.8,0.0


This has now put the Easting and Northing values in a form that is mathematically equivalent to what they were previously, but are now encoded to protect the actual locations.

## Combine date and time

The date and time fields are combined into one named DateTime and the resulting column is converted into a timestamp.

In [5]:
from datetime import datetime

telemetry['DateTime'] = telemetry['Date'] + ' ' + telemetry['Time']

def convert_to_unix(col):
	d = datetime.strptime(col, '%y-%m-%d %H:%M:%S.%f')
	return d.timestamp() * 1000

telemetry['DateTime'] = telemetry['DateTime'].map(convert_to_unix)
telemetry.head(5)

,Date,Time,Easting,Northing,WaterDepth,Roll,Pitch,Heading,Tide,DateTime
0,20-02-27,20:50:47.502,0.158187,0.410159,0.619607,-6.3,2.0,19.9,0.0,1.582837e+12
1,20-02-27,20:50:47.502,0.158187,0.410159,0.038946,-6.3,2.0,19.9,0.0,1.582837e+12
2,20-02-27,20:50:49.416,0.534407,0.535898,0.038946,-5.0,3.1,22.8,0.0,1.582837e+12
3,20-02-27,20:50:49.416,0.534311,0.535924,0.042081,-5.0,3.1,22.8,0.0,1.582837e+12
4,20-02-27,20:50:49.416,0.533734,0.536254,0.046682,-5.0,3.1,22.8,0.0,1.582837e+12


We can then remove the unneeded date and time columns:s

In [6]:
telemetry.drop(columns=['Date', 'Time'], inplace=True)
telemetry['DateTimeNorm'] = normalise('DateTime')

## Empty Values
The Tide column appears at first glance to consist only of values of `0.0`. We can check this by finding the unique values:

In [7]:
telemetry['Tide'].unique()

array([0.])

This column only contains 0.0, which won't be useful to us, so we can drop this column.

In [8]:
# Drop unnecessary fields
telemetry.drop(columns=['Tide'], inplace=True)

In [9]:
telemetry.head(5)

,Easting,Northing,WaterDepth,Roll,Pitch,Heading,DateTime,DateTimeNorm
0,0.158187,0.410159,0.619607,-6.3,2.0,19.9,1.582837e+12,0.000000
1,0.158187,0.410159,0.038946,-6.3,2.0,19.9,1.582837e+12,0.000000
2,0.534407,0.535898,0.038946,-5.0,3.1,22.8,1.582837e+12,0.000004
3,0.534311,0.535924,0.042081,-5.0,3.1,22.8,1.582837e+12,0.000004
4,0.533734,0.536254,0.046682,-5.0,3.1,22.8,1.582837e+12,0.000004


## Write to CSV
Now that we have processed the telemetry, we can write the dataframe to a file. Previously the format was ETR, but since this is equivalent to a space-delimited CSV, we have opted to write the data to a CSV to put it in a more commonly understood format.

In [10]:
telemetry.to_csv('../Api/src/data/telemetry.csv', index=False)
telemetry.to_csv('./telemetry_cleaned.csv', index=False)